# �� Diagnóstico Profundo del Dataset

**Objetivo:** Auditoría estadística completa del dataset de entrenamiento para identificar:
1. Tipos de datos (continuo vs discreto)
2. Cardinalidad de variables
3. Tasa de censura
4. Restricciones de dominio
5. Problemas potenciales

---

## Justificación Metodológica

> Según **Lawless (2003)**, *"Statistical Models and Methods for Lifetime Data"*:
> - La distribución del tiempo al evento debe examinarse antes de modelar
> - Datos de intervalo (interval-censored) requieren tratamiento especial
> - La tasa de censura afecta la potencia estadística

---

In [1]:
# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
from pathlib import Path

DATA_DIR = Path("data/processed")
TRAIN_PATH = DATA_DIR / "train_final.parquet"  # Usamos train_final ya que train_survival_selected no existe

print("✅ Configuración cargada")
print(f"   Dataset: {TRAIN_PATH}")

✅ Configuración cargada
   Dataset: data/processed/train_final.parquet


In [2]:
# ==============================================================================
# 1. CARGA Y OVERVIEW
# ==============================================================================
df = pd.read_parquet(TRAIN_PATH)

print("=" * 70)
print("📊 OVERVIEW DEL DATASET")
print("=" * 70)
print(f"\nDimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"\nColumnas:")
for i, c in enumerate(df.columns):
    print(f"   [{i:2d}] {c}")

📊 OVERVIEW DEL DATASET

Dimensiones: 296 filas × 63 columnas

Columnas:
   [ 0] edad
   [ 1] genero_m
   [ 2] hab_1
   [ 3] hab_2
   [ 4] hab_3
   [ 5] hab_4
   [ 6] hab_5
   [ 7] hab_6
   [ 8] hab_7
   [ 9] tech_programacion
   [10] tech_python
   [11] tech_java
   [12] tech_desarrollo_web
   [13] tech_desarrollo_movil
   [14] tech_base_datos
   [15] tech_machine_learning
   [16] tech_inteligencia_artificial
   [17] tech_analisis_datos
   [18] tech_big_data
   [19] tech_redes
   [20] tech_telecomunicaciones
   [21] tech_redes_inalambricas
   [22] tech_sistemas_celulares
   [23] tech_ciberseguridad
   [24] tech_cloud_computing
   [25] tech_sistemas_operativos
   [26] tech_iot
   [27] tech_automatizacion
   [28] tech_robotica
   [29] tech_electronica
   [30] tech_instrumentacion
   [31] tech_control
   [32] tech_electricidad
   [33] tech_sistemas_potencia
   [34] tech_energias_renovables
   [35] tech_calidad_energia
   [36] tech_termodinamica
   [37] tech_mecanica_fluidos
   [38] tech_d

---
## 2. Análisis de Tipos de Datos

In [3]:
# ==============================================================================
# CLASIFICACIÓN DE COLUMNAS
# ==============================================================================

# Identificar tipos
continuous_cols = []
categorical_cols = []
binary_cols = []

for col in df.columns:
    if col in ['event', 'duration']:
        continue  # Targets
    
    unique_vals = df[col].nunique()
    dtype = df[col].dtype
    
    if unique_vals == 2:
        binary_cols.append(col)
        categorical_cols.append(col)
    elif unique_vals <= 10 or col.startswith('tech_'):
        categorical_cols.append(col)
    else:
        continuous_cols.append(col)

print("=" * 70)
print("📋 CLASIFICACIÓN DE VARIABLES")
print("=" * 70)

print(f"\n🔢 CONTINUAS ({len(continuous_cols)}):")
for c in continuous_cols:
    print(f"   - {c}: {df[c].nunique()} valores únicos, rango [{df[c].min():.2f}, {df[c].max():.2f}]")

print(f"\n📊 CATEGÓRICAS ({len(categorical_cols)}):")
print(f"   - Binarias: {len(binary_cols)}")
cat_non_binary = [c for c in categorical_cols if c not in binary_cols]
print(f"   - Multi-clase: {len(cat_non_binary)}")

📋 CLASIFICACIÓN DE VARIABLES

🔢 CONTINUAS (1):
   - edad: 17 valores únicos, rango [21.00, 40.00]

📊 CATEGÓRICAS (60):
   - Binarias: 51
   - Multi-clase: 9


---
## 3. Análisis del Target (Event/Duration)

In [4]:
# ==============================================================================
# ANÁLISIS DE CENSURA Y DURACIÓN
# ==============================================================================

print("=" * 70)
print("🎯 ANÁLISIS DEL TARGET")
print("=" * 70)

# Event (censura)
n_events = df['event'].sum()
n_censored = len(df) - n_events
censoring_rate = n_censored / len(df)

print(f"\n📌 EVENTO (event):")
print(f"   Eventos (E=1):    {n_events} ({100*n_events/len(df):.1f}%)")
print(f"   Censurados (E=0): {n_censored} ({100*censoring_rate:.1f}%)")
print(f"   Tasa de censura:  {censoring_rate:.4f}")

# Duration
print(f"\n⏱️ DURACIÓN (duration):")
print(f"   Mínimo:  {df['duration'].min():.2f} meses")
print(f"   Máximo:  {df['duration'].max():.2f} meses")
print(f"   Media:   {df['duration'].mean():.2f} meses")
print(f"   Mediana: {df['duration'].median():.2f} meses")
print(f"   Std:     {df['duration'].std():.2f}")
print(f"   Valores únicos: {df['duration'].nunique()}")

# Distribución por evento
print(f"\n📊 DISTRIBUCIÓN POR EVENTO:")
print(f"   Duration (E=0): mean={df[df['event']==0]['duration'].mean():.2f}, "
      f"std={df[df['event']==0]['duration'].std():.2f}")
print(f"   Duration (E=1): mean={df[df['event']==1]['duration'].mean():.2f}, "
      f"std={df[df['event']==1]['duration'].std():.2f}")

🎯 ANÁLISIS DEL TARGET

📌 EVENTO (event):
   Eventos (E=1):    135 (45.6%)
   Censurados (E=0): 161 (54.4%)
   Tasa de censura:  0.5439

⏱️ DURACIÓN (duration):
   Mínimo:  0.58 meses
   Máximo:  30.00 meses
   Media:   15.43 meses
   Mediana: 17.64 meses
   Std:     10.99
   Valores únicos: 265

📊 DISTRIBUCIÓN POR EVENTO:
   Duration (E=0): mean=10.37, std=10.92
   Duration (E=1): mean=21.47, std=7.48


---
## 4. Detección de Restricciones de Dominio

In [5]:
# ==============================================================================
# RESTRICCIONES LÓGICAS
# ==============================================================================

constraints = []

print("=" * 70)
print("🔒 RESTRICCIONES DE DOMINIO DETECTADAS")
print("=" * 70)

# 1. Duration > 0
if (df['duration'] > 0).all():
    constraints.append({'variable': 'duration', 'constraint': 'duration > 0', 'status': 'OK'})
    print("\n✅ duration > 0: Cumplida")
else:
    constraints.append({'variable': 'duration', 'constraint': 'duration > 0', 'status': 'VIOLATED'})
    print("\n❌ duration > 0: Violada")

# 2. Event binario
if set(df['event'].unique()) <= {0, 1}:
    constraints.append({'variable': 'event', 'constraint': 'event ∈ {0, 1}', 'status': 'OK'})
    print("✅ event ∈ {0, 1}: Cumplida")
else:
    constraints.append({'variable': 'event', 'constraint': 'event ∈ {0, 1}', 'status': 'VIOLATED'})
    print("❌ event ∈ {0, 1}: Violada")

# 3. Edad razonable
if 'edad' in df.columns:
    edad_min, edad_max = df['edad'].min(), df['edad'].max()
    if edad_min >= 18 and edad_max <= 65:
        constraints.append({'variable': 'edad', 'constraint': '18 ≤ edad ≤ 65', 'status': 'OK'})
        print(f"✅ Edad razonable [{edad_min:.0f}, {edad_max:.0f}]: Cumplida")
    else:
        constraints.append({'variable': 'edad', 'constraint': '18 ≤ edad ≤ 65', 'status': 'WARNING'})
        print(f"⚠️ Edad [{edad_min:.0f}, {edad_max:.0f}]: Revisar valores extremos")

# 4. Soft skills [0, 1]
hab_cols = [c for c in df.columns if c.startswith('hab_')]
for h in hab_cols:
    if df[h].min() >= 0 and df[h].max() <= 1:
        constraints.append({'variable': h, 'constraint': '0 ≤ value ≤ 1', 'status': 'OK'})
    else:
        constraints.append({'variable': h, 'constraint': '0 ≤ value ≤ 1', 'status': 'WARNING'})
        
print(f"✅ Soft skills normalizadas [0,1]: {len(hab_cols)} variables")

# 5. Tech skills binarias
tech_cols = [c for c in df.columns if c.startswith('tech_')]
tech_binary_ok = all(set(df[c].unique()) <= {0, 1} for c in tech_cols)
if tech_binary_ok:
    constraints.append({'variable': 'tech_*', 'constraint': 'value ∈ {0, 1}', 'status': 'OK'})
    print(f"✅ Tech skills binarias: {len(tech_cols)} variables")
else:
    constraints.append({'variable': 'tech_*', 'constraint': 'value ∈ {0, 1}', 'status': 'WARNING'})
    print(f"⚠️ Tech skills: Algunos valores no binarios")

🔒 RESTRICCIONES DE DOMINIO DETECTADAS

✅ duration > 0: Cumplida
✅ event ∈ {0, 1}: Cumplida
✅ Edad razonable [21, 40]: Cumplida
✅ Soft skills normalizadas [0,1]: 7 variables
✅ Tech skills binarias: 52 variables


In [6]:
# ==============================================================================
# DETECCIÓN DE PII (Información Personal Identificable)
# ==============================================================================

print("\n" + "=" * 70)
print("🔐 ANÁLISIS DE PII (Privacidad)")
print("=" * 70)

pii_candidates = []

# Buscar columnas que podrían contener PII
pii_keywords = ['nombre', 'email', 'telefono', 'cedula', 'id', 'direccion', 
                'name', 'mail', 'phone', 'address', 'ssn']

for col in df.columns:
    col_lower = col.lower()
    for kw in pii_keywords:
        if kw in col_lower:
            pii_candidates.append(col)
            break

if pii_candidates:
    print(f"\n⚠️ Posibles columnas PII detectadas: {pii_candidates}")
else:
    print("\n✅ No se detectaron columnas PII obvias")
    print("   (El dataset parece anonimizado)")


🔐 ANÁLISIS DE PII (Privacidad)

⚠️ Posibles columnas PII detectadas: ['tech_ciberseguridad', 'tech_electricidad', 'tech_calidad_energia', 'tech_mecanica_fluidos', 'tech_gestion_calidad', 'tech_hidraulica_sanitaria']


---
## 5. Análisis de Correlaciones

In [7]:
# ==============================================================================
# CORRELACIONES CON EL TARGET
# ==============================================================================

print("=" * 70)
print("📈 CORRELACIONES CON EVENT")
print("=" * 70)

# Solo columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
corrs = df[numeric_cols].corr()['event'].drop(['event', 'duration']).sort_values(key=abs, ascending=False)

print("\nTop 10 correlaciones con event:")
for i, (col, corr) in enumerate(corrs.head(10).items(), 1):
    star = "⭐" if abs(corr) > 0.1 else ""
    print(f"   {i:2d}. {col:30s}: {corr:+.4f} {star}")

print(f"\n📌 Variables con |r| > 0.1: {sum(abs(corrs) > 0.1)}")
print(f"📌 Variables con |r| > 0.2: {sum(abs(corrs) > 0.2)}")

# Correlación duration-event
corr_de = df[['duration', 'event']].corr().iloc[0,1]
print(f"\n🔗 Correlación duration-event: {corr_de:.4f}")

📈 CORRELACIONES CON EVENT

Top 10 correlaciones con event:
    1. tech_sistemas_operativos      : +0.1697 ⭐
    2. hab_1                         : +0.1511 ⭐
    3. tech_geotecnia                : -0.1440 ⭐
    4. tech_gestion_empresarial      : +0.1402 ⭐
    5. tech_gestion_calidad          : +0.1298 ⭐
    6. tech_programacion             : +0.1290 ⭐
    7. tech_mecanica_fluidos         : -0.1229 ⭐
    8. tech_electricidad             : +0.1213 ⭐
    9. hab_7                         : +0.1159 ⭐
   10. tech_algoritmos               : +0.1144 ⭐

📌 Variables con |r| > 0.1: 17
📌 Variables con |r| > 0.2: 0

🔗 Correlación duration-event: 0.5040


---
## 6. Diagnóstico del Problema de Modelado

In [8]:
# ==============================================================================
# DIAGNÓSTICO: ¿POR QUÉ FALLA EL MODELO?
# ==============================================================================

print("=" * 70)
print("🔬 DIAGNÓSTICO: ¿POR QUÉ FALLAN LOS MODELOS?")
print("=" * 70)

problems = []

# 1. Correlaciones muy bajas
max_corr = abs(corrs).max()
if max_corr < 0.2:
    problems.append("🚨 CRÍTICO: Ninguna feature tiene correlación > 0.2 con el target")
    print(f"\n1. PROBLEMA: Correlaciones muy bajas")
    print(f"   Máxima correlación con event: {max_corr:.4f}")
    print(f"   → Las features no tienen poder predictivo")

# 2. Alta correlación duration-event
if abs(corr_de) > 0.4:
    problems.append("⚠️ Alta correlación duration-event")
    print(f"\n2. OBSERVACIÓN: Correlación duration-event alta ({corr_de:.4f})")

# 3. Dataset pequeño
if len(df) < 500:
    problems.append("⚠️ Dataset pequeño (N < 500)")
    print(f"\n3. LIMITACIÓN: Dataset pequeño (N={len(df)})")
    print(f"   → Difícil detectar patrones débiles")

# 4. Alta dimensionalidad
n_features = len([c for c in df.columns if c not in ['event', 'duration']])
if n_features > len(df) / 5:
    problems.append("⚠️ Alta dimensionalidad relativa")
    print(f"\n4. PROBLEMA: Alta dimensionalidad")
    print(f"   Features: {n_features}, N/p ratio: {len(df)/n_features:.1f}")

# Resumen
print(f"\n📋 RESUMEN DE PROBLEMAS: {len(problems)}")
for p in problems:
    print(f"   • {p}")

🔬 DIAGNÓSTICO: ¿POR QUÉ FALLAN LOS MODELOS?

1. PROBLEMA: Correlaciones muy bajas
   Máxima correlación con event: 0.1697
   → Las features no tienen poder predictivo

2. OBSERVACIÓN: Correlación duration-event alta (0.5040)

3. LIMITACIÓN: Dataset pequeño (N=296)
   → Difícil detectar patrones débiles

4. PROBLEMA: Alta dimensionalidad
   Features: 61, N/p ratio: 4.9

📋 RESUMEN DE PROBLEMAS: 4
   • 🚨 CRÍTICO: Ninguna feature tiene correlación > 0.2 con el target
   • ⚠️ Alta correlación duration-event
   • ⚠️ Dataset pequeño (N < 500)
   • ⚠️ Alta dimensionalidad relativa


In [9]:
# ==============================================================================
# EXPORTAR REPORTE JSON
# ==============================================================================

report = {
    'n_rows': int(len(df)),
    'n_cols': int(len(df.columns)),
    'censoring_rate': float(censoring_rate),
    'event_rate': float(1 - censoring_rate),
    'continuous_cols': continuous_cols,
    'categorical_cols': categorical_cols,
    'binary_cols': binary_cols,
    'constraints_detected': constraints,
    'max_correlation_with_event': float(max_corr),
    'duration_range': [float(df['duration'].min()), float(df['duration'].max())],
    'duration_unique_values': int(df['duration'].nunique()),
    'pii_detected': pii_candidates,
    'problems_identified': problems
}

with open(DATA_DIR / 'diagnosis_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\n" + "=" * 70)
print("📁 REPORTE GUARDADO")
print("=" * 70)
print(f"\nArchivo: {DATA_DIR / 'diagnosis_report.json'}")
print(f"\n📊 Resumen:")
print(f"   • Filas: {report['n_rows']}")
print(f"   • Columnas: {report['n_cols']}")
print(f"   • Tasa de censura: {report['censoring_rate']:.2%}")
print(f"   • Vars continuas: {len(report['continuous_cols'])}")
print(f"   • Vars categóricas: {len(report['categorical_cols'])}")
print(f"   • Max correlación: {report['max_correlation_with_event']:.4f}")


📁 REPORTE GUARDADO

Archivo: data/processed/diagnosis_report.json

📊 Resumen:
   • Filas: 296
   • Columnas: 63
   • Tasa de censura: 54.39%
   • Vars continuas: 1
   • Vars categóricas: 60
   • Max correlación: 0.1697
